# Finetuning using IndoBART

In [1]:
! git clone https://github.com/indobenchmark/indonlg.git

Cloning into 'indonlg'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 152 (delta 75), reused 97 (delta 34), pack-reused 0
Receiving objects: 100% (152/152), 2.32 MiB | 9.81 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [2]:
%cd /content/indonlg
! pip install -r requirements.txt

/content/indonlg
     |████████████████████████████████| 1.5 MB 14.5 MB/s 
     |████████████████████████████████| 68 kB 3.0 MB/s 
     |████████████████████████████████| 776.8 MB 16 kB/s 
     |████████████████████████████████| 186 kB 68.5 MB/s 
     |████████████████████████████████| 3.3 MB 50.5 MB/s 
     |████████████████████████████████| 394.7 MB 18 kB/s 
     |████████████████████████████████| 1.2 MB 64.1 MB/s 
     |████████████████████████████████| 22.3 MB 800 kB/s 
     |████████████████████████████████| 65 kB 4.8 MB/s 
     |████████████████████████████████| 149 kB 102.7 MB/s 
     |████████████████████████████████| 1.8 MB 41.1 MB/s 
     |████████████████████████████████| 212 kB 85.3 MB/s 
     |████████████████████████████████| 136 kB 68.2 MB/s 
     |████████████████████████████████| 2.9 MB 49.6 MB/s 
     |████████████████████████████████| 3.8 MB 60.3 MB/s 
     |████████████████████████████████| 14.8 MB 57.5 MB/s 
     |████████████████████████████████| 132 kB 99.6 MB/s 

In [1]:
%cd /content/indonlg/examples

/content/indonlg/examples


In [2]:
import os, sys
sys.path.append('../')
os.chdir('../')

import torch
import shutil
import random
import numpy as np
import pandas as pd
from torch import optim
from torch.utils.data import ConcatDataset
from transformers import MBartForConditionalGeneration

from indobenchmark import IndoNLGTokenizer
from utils.train_eval import train, evaluate
from utils.metrics import generation_metrics_fn
from utils.forward_fn import forward_generation
from utils.data_utils import MachineTranslationDataset, GenerationDataLoader

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
# Set random seed
# set_seed(26092020)

# Load Model

In [5]:
bart_model = MBartForConditionalGeneration.from_pretrained('indobenchmark/indobart-v2')
tokenizer = IndoNLGTokenizer.from_pretrained('indobenchmark/indobart-v2')

model = bart_model
model

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(40004, 768, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(40004, 768, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 768, padding_idx=1)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=Tru

In [6]:
count_param(model)

131543040

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Test model to generate sequences

In [ ]:
inputs = ['aku pergi ke toko obat membeli <mask>']
bart_input = tokenizer.prepare_input_for_generation(inputs, return_tensors='pt',
                                         lang_token = '[indonesian]', decoder_lang_token='[indonesian]')

bart_input.to(device)
bart_out = model(**bart_input)
print(tokenizer.decode(bart_input['input_ids'][0]))
print(tokenizer.decode(bart_out.logits.topk(1).indices[:,:].squeeze()))

In [ ]:
inputs = ['aku menyang pasar karo <mask>']
bart_input = tokenizer.prepare_input_for_generation(inputs, return_tensors='pt',
                                         lang_token = '[javanese]', decoder_lang_token='[javanese]')

bart_input.to(device)
bart_out = bart_model(**bart_input)
print(tokenizer.decode(bart_input['input_ids'][0]))
print(tokenizer.decode(bart_out.logits.topk(1).indices[:,:].squeeze()))

<s> aku menyang pasar karo <mask> [javanese]
<s> aku menyang pasar karo tuku </s>


In [ ]:
inputs = ['kuring ka pasar senen meuli daging <mask>']
bart_input = tokenizer.prepare_input_for_generation(inputs, return_tensors='pt',
                                         lang_token = '[sundanese]', decoder_lang_token='[sundanese]')

bart_input.to(device)
bart_out = bart_model(**bart_input)
print(tokenizer.decode(bart_input['input_ids'][0]))
print(tokenizer.decode(bart_out.logits.topk(1).indices[:,:].squeeze()))

<s> kuring ka pasar senen meuli daging <mask> [sundanese]
<s> kuring ka pasar senen meuli daging sapi.


# Multilingual Model

## Prepare Dataset

In [8]:
!pwd

/content/indonlg


In [9]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = False
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 8
valid_batch_size = 8
test_batch_size = 8

source_lang = "[indonesian]"
target_lang = "[indonesian]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/multi/example'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [10]:
train_dataset_su_path = '/content/indonlg/dataset/MT_SUNIBS_INZNTV/train_preprocess.json'
valid_dataset_su_path = '/content/indonlg/dataset/MT_SUNIBS_INZNTV/valid_preprocess.json'
test_dataset_su_path = '/content/indonlg/dataset/MT_SUNIBS_INZNTV/test_preprocess.json'

train_dataset_su = MachineTranslationDataset(train_dataset_su_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset_su = MachineTranslationDataset(valid_dataset_su_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset_su = MachineTranslationDataset(test_dataset_su_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_dataset_min_path = '/content/drive/MyDrive/11-737/FinalProject/data/min-id/train_preprocess.json'
valid_dataset_min_path = '/content/drive/MyDrive/11-737/FinalProject/data/min-id/valid_preprocess.json'
test_dataset_min_path = '/content/drive/MyDrive/11-737/FinalProject/data/min-id/test_preprocess.json'

train_dataset_min = MachineTranslationDataset(train_dataset_min_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset_min = MachineTranslationDataset(valid_dataset_min_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset_min = MachineTranslationDataset(test_dataset_min_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_dataset_jv_path = '/content/indonlg/dataset/MT_JAVNRF_INZNTV/train_preprocess.json'
valid_dataset_jv_path = '/content/indonlg/dataset/MT_JAVNRF_INZNTV/valid_preprocess.json'
test_dataset_jv_path = '/content/indonlg/dataset/MT_JAVNRF_INZNTV/test_preprocess.json'

train_dataset_jv = MachineTranslationDataset(train_dataset_jv_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset_jv = MachineTranslationDataset(valid_dataset_jv_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset_jv = MachineTranslationDataset(test_dataset_jv_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_dataset = ConcatDataset([train_dataset_su, train_dataset_jv, train_dataset_min])
valid_dataset = ConcatDataset([valid_dataset_su, valid_dataset_jv, valid_dataset_min])
test_dataset = ConcatDataset([test_dataset_su, test_dataset_jv, test_dataset_min])

train_loader = GenerationDataLoader(dataset=train_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

## Fine Tuning on all

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:1.1689 LR:0.00010000: 100%|██████████| 2939/2939 [09:02<00:00,  5.42it/s]


(Epoch 1) TRAIN LOSS:1.1689 BLEU:62.37 SacreBLEU:63.50 ROUGE1:77.59 ROUGE2:60.40 ROUGEL:75.95 ROUGELsum:75.95 LR:0.00010000


VALID LOSS:1.2103: 100%|██████████| 400/400 [00:22<00:00, 18.02it/s]


(Epoch 1) VALID LOSS:1.2103 BLEU:55.51 SacreBLEU:55.63 ROUGE1:75.30 ROUGE2:58.11 ROUGEL:73.74 ROUGELsum:73.73


(Epoch 2) TRAIN LOSS:0.7764 LR:0.00009000: 100%|██████████| 2939/2939 [09:04<00:00,  5.40it/s]


...

(Epoch 9) VALID LOSS:1.1483 BLEU:65.06 SacreBLEU:65.18 ROUGE1:81.07 ROUGE2:67.19 ROUGEL:80.01 ROUGELsum:80.01


(Epoch 10) TRAIN LOSS:0.0478 LR:0.00003874: 100%|██████████| 2939/2939 [09:02<00:00,  5.42it/s]


(Epoch 10) TRAIN LOSS:0.0478 BLEU:97.43 SacreBLEU:97.51 ROUGE1:98.94 ROUGE2:97.85 ROUGEL:98.93 ROUGELsum:98.93 LR:0.00003874


VALID LOSS:1.1600: 100%|██████████| 400/400 [00:22<00:00, 17.82it/s]


(Epoch 10) VALID LOSS:1.1600 BLEU:65.61 SacreBLEU:65.73 ROUGE1:81.34 ROUGE2:67.82 ROUGEL:80.40 ROUGELsum:80.40


In [ ]:
! cp '/content/indonlg/save/multi/example/best_model_0.th' '/content/drive/My Drive/'

## jv-id



In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/drive/My Drive/11-737/FinalProject/models/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = False
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 8
valid_batch_size = 8
test_batch_size = 8

source_lang = "[javanese]"
target_lang = "[indonesian]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/jv-id/example'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:

train_loader = GenerationDataLoader(dataset=train_dataset_jv, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_jv, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_jv, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
# Train

n_epochs = 20

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:0.1319 LR:0.00010000: 100%|██████████| 746/746 [04:42<00:00,  2.64it/s]


(Epoch 1) TRAIN LOSS:0.1319 BLEU:91.98 SacreBLEU:92.28 ROUGE1:96.25 ROUGE2:92.47 ROUGEL:96.20 ROUGELsum:96.20 LR:0.00010000


VALID LOSS:1.1770: 100%|██████████| 100/100 [00:12<00:00,  8.15it/s]


(Epoch 1) VALID LOSS:1.1770 BLEU:55.07 SacreBLEU:55.30 ROUGE1:76.52 ROUGE2:58.20 ROUGEL:75.45 ROUGELsum:75.46


(Epoch 2) TRAIN LOSS:0.1140 LR:0.00009000: 100%|██████████| 746/746 [04:37<00:00,  2.68it/s]


...

(Epoch 5) VALID LOSS:1.3024 BLEU:53.69 SacreBLEU:53.93 ROUGE1:76.21 ROUGE2:57.05 ROUGEL:74.85 ROUGELsum:74.86
count stop: 4


(Epoch 6) TRAIN LOSS:0.0308 LR:0.00005905: 100%|██████████| 746/746 [04:37<00:00,  2.68it/s]


(Epoch 6) TRAIN LOSS:0.0308 BLEU:98.31 SacreBLEU:98.38 ROUGE1:99.28 ROUGE2:98.48 ROUGEL:99.28 ROUGELsum:99.28 LR:0.00005905


VALID LOSS:1.2705: 100%|██████████| 100/100 [00:12<00:00,  8.27it/s]


(Epoch 6) VALID LOSS:1.2705 BLEU:54.37 SacreBLEU:54.63 ROUGE1:76.46 ROUGE2:57.67 ROUGEL:75.23 ROUGELsum:75.25
count stop: 5


In [ ]:
! cp '/content/indonlg/save/jv-id/example/best_model_0.th' '/content/drive/My Drive/best_model_multi_jv_id'

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/indonlg/save/jv-id/example/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 150/150 [03:57<00:00,  1.58s/it]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result.csv")

== Prediction Result ==
                                                 hyp  \
0  pada waktu itu her odes, raja wilayah, mendeng...   
1  " dan dalam keadaan yang demikian, ketika aku ...   
2  dan karena itu ia tahu sebelum ia mengetahui a...   
3             kasih karunia menyertai kamu sekalian.   
4  dengan tanganku sendiri aku melakukan pekerjaa...   

                                               label  
0  pada masa itu sampailah berita - berita tentan...  
1  " dan dalam keadaan demikian, ketika aku denga...  
2  karena itu ia telah melihat ke depan dan telah...  
3             kasih karunia menyertai kamu sekalian.  
4  kami melakukan pekerjaan tangan yang berat. ka...  

== Model Performance ==
            BLEU  SacreBLEU    ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.000000   1.000000   1.00000   1.000000   1.000000   1.000000
mean   44.883209  45.110281  66.65173  47.923635  63.929153  63.921786
std          NaN        NaN       NaN        NaN        NaN        NaN


## su-id



In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = False
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 8
valid_batch_size = 8
test_batch_size = 8

source_lang = "[sundanese]"
target_lang = "[indonesian]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/su-id/example'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:

train_loader = GenerationDataLoader(dataset=train_dataset_su, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_su, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_su, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:0.1686 LR:0.00010000: 100%|██████████| 746/746 [04:42<00:00,  2.64it/s]


(Epoch 1) TRAIN LOSS:0.1686 BLEU:89.65 SacreBLEU:90.00 ROUGE1:95.41 ROUGE2:90.55 ROUGEL:95.33 ROUGELsum:95.33 LR:0.00010000


VALID LOSS:2.1809: 100%|██████████| 100/100 [00:12<00:00,  7.98it/s]


(Epoch 1) VALID LOSS:2.1809 BLEU:34.59 SacreBLEU:34.92 ROUGE1:61.77 ROUGE2:37.10 ROUGEL:58.89 ROUGELsum:58.87


(Epoch 2) TRAIN LOSS:0.1217 LR:0.00009000: 100%|██████████| 746/746 [04:42<00:00,  2.64it/s]


...

(Epoch 9) VALID LOSS:2.4166 BLEU:35.11 SacreBLEU:35.47 ROUGE1:62.40 ROUGE2:37.12 ROUGEL:59.24 ROUGELsum:59.23
count stop: 2


(Epoch 10) TRAIN LOSS:0.0170 LR:0.00003874: 100%|██████████| 746/746 [04:42<00:00,  2.64it/s]


(Epoch 10) TRAIN LOSS:0.0170 BLEU:99.16 SacreBLEU:99.19 ROUGE1:99.68 ROUGE2:99.30 ROUGEL:99.68 ROUGELsum:99.68 LR:0.00003874


VALID LOSS:2.4606: 100%|██████████| 100/100 [00:12<00:00,  7.97it/s]


(Epoch 10) VALID LOSS:2.4606 BLEU:34.82 SacreBLEU:35.24 ROUGE1:62.25 ROUGE2:37.22 ROUGEL:59.26 ROUGELsum:59.22
count stop: 3


In [ ]:
! cp '/content/indonlg/save/su-id/example/best_model_0.th' '/content/drive/My Drive/best_model_multi_su_id'

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/indonlg/save/su-id/example/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 150/150 [03:31<00:00,  1.41s/it]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_su.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_su.csv")

== Prediction Result ==
                                                 hyp  \
0  sebab aku tidak tahu apa yang telah terjadi, d...   
1          dan mereka semuanya makan sampai kenyang.   
2  karena kita adalah bait allah, yang telah dici...   
3  bagi yang kudus segala sesuatu adalah kudus, t...   
4  setelah lepas lah petrus dan yohanes mengunjun...   

                                               label  
0  sebab memang aku tidak sadar akan sesuatu, tet...  
1          dan mereka semuanya makan sampai kenyang.  
2  karena kita ini buatan allah, diciptakan dalam...  
3  bagi orang suci semuanya suci ; tetapi bagi or...  
4  sesudah dilepaskan pergilah petrus dan yohanes...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1     ROUGE2    ROUGEL  ROUGELsum
count   1.000000   1.000000   1.000000   1.000000   1.00000   1.000000
mean   23.132595  23.433673  46.674672  25.452321  41.74109  41.718037
std          NaN        NaN        NaN        NaN       NaN        NaN


## min-id



In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = False
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 8
valid_batch_size = 8
test_batch_size = 8

source_lang = "[indonesian]"
target_lang = "[indonesian]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/min-id/example'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:

train_loader = GenerationDataLoader(dataset=train_dataset_min, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_min, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_min, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:0.1155 LR:0.00010000: 100%|██████████| 1447/1447 [09:47<00:00,  2.46it/s]


(Epoch 1) TRAIN LOSS:0.1155 BLEU:94.30 SacreBLEU:94.43 ROUGE1:97.62 ROUGE2:95.15 ROUGEL:97.58 ROUGELsum:97.58 LR:0.00010000


VALID LOSS:0.9351: 100%|██████████| 200/200 [00:25<00:00,  7.70it/s]


(Epoch 1) VALID LOSS:0.9351 BLEU:77.47 SacreBLEU:77.49 ROUGE1:87.67 ROUGE2:78.43 ROUGEL:87.42 ROUGELsum:87.40


(Epoch 2) TRAIN LOSS:0.0984 LR:0.00009000: 100%|██████████| 1447/1447 [09:44<00:00,  2.48it/s]


...

(Epoch 9) VALID LOSS:0.9854 BLEU:79.36 SacreBLEU:79.38 ROUGE1:88.65 ROUGE2:80.22 ROUGEL:88.46 ROUGELsum:88.44
count stop: 1


(Epoch 10) TRAIN LOSS:0.0141 LR:0.00003874: 100%|██████████| 1447/1447 [09:42<00:00,  2.48it/s]


(Epoch 10) TRAIN LOSS:0.0141 BLEU:99.33 SacreBLEU:99.35 ROUGE1:99.77 ROUGE2:99.52 ROUGEL:99.77 ROUGELsum:99.77 LR:0.00003874


VALID LOSS:0.9990: 100%|██████████| 200/200 [00:25<00:00,  7.78it/s]


(Epoch 10) VALID LOSS:0.9990 BLEU:79.83 SacreBLEU:79.85 ROUGE1:88.82 ROUGE2:80.56 ROUGEL:88.61 ROUGELsum:88.61


In [ ]:
! cp '/content/indonlg/save/min-id/example/best_model_0.th' '/content/drive/My Drive/best_model_multi_min_id'

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/indonlg/save/min-id/example/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 400/400 [12:04<00:00,  1.81s/it]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_min.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_min.csv")

== Prediction Result ==
                                                 hyp  \
0  kehadiran & qu ot ; bo eka mata & qu ; menyeba...   
1  sop hr onica o ch re ic eps adalah spesies kum...   
2  umumnya manusia dengan mudah menerima prinsip ...   
3  bagian kode yang melakukan tugas  <0x40> - <0x...   
4  namun pernikahan tersebut tidak berakhir denga...   

                                               label  
0  kehadiran & qu ot ; bo eka mata & qu ot ; meny...  
1  sop hr onica o ch re ic eps adalah spesies kum...  
2  umumnya manusia dengan mudah menerima prinsip ...  
3  bagian kode yang melakukan tugas  <0x40> - <0x...  
4  namun pernikahan tersebut tidak berakhir denga...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1     ROUGE2    ROUGEL  ROUGELsum
count   1.000000   1.000000   1.000000   1.000000   1.00000   1.000000
mean   72.890057  72.904393  86.668518  77.690252  86.28497  86.292981
std          NaN        NaN        NaN        NaN       NaN        NaN


## id-jv



In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = True
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 8
valid_batch_size = 8
test_batch_size = 8

source_lang = "[indonesian]"
target_lang = "[javanese]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/id-jv/example'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:

train_loader = GenerationDataLoader(dataset=train_dataset_jv, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_jv, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_jv, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:0.1348 LR:0.00010000: 100%|██████████| 746/746 [04:48<00:00,  2.59it/s]


(Epoch 1) TRAIN LOSS:0.1348 BLEU:91.98 SacreBLEU:92.30 ROUGE1:96.26 ROUGE2:92.48 ROUGEL:96.22 ROUGELsum:96.21 LR:0.00010000


VALID LOSS:1.1902: 100%|██████████| 100/100 [00:12<00:00,  8.18it/s]


(Epoch 1) VALID LOSS:1.1902 BLEU:54.60 SacreBLEU:54.83 ROUGE1:76.56 ROUGE2:58.04 ROUGEL:75.37 ROUGELsum:75.35


(Epoch 2) TRAIN LOSS:0.1149 LR:0.00009000: 100%|██████████| 746/746 [04:36<00:00,  2.70it/s]


...

(Epoch 5) VALID LOSS:1.2911 BLEU:53.84 SacreBLEU:54.10 ROUGE1:75.99 ROUGE2:57.01 ROUGEL:74.87 ROUGELsum:74.87
count stop: 4


(Epoch 6) TRAIN LOSS:0.0307 LR:0.00005905: 100%|██████████| 746/746 [04:35<00:00,  2.71it/s]


(Epoch 6) TRAIN LOSS:0.0307 BLEU:98.33 SacreBLEU:98.40 ROUGE1:99.30 ROUGE2:98.50 ROUGEL:99.29 ROUGELsum:99.29 LR:0.00005905


VALID LOSS:1.3015: 100%|██████████| 100/100 [00:12<00:00,  8.15it/s]


(Epoch 6) VALID LOSS:1.3015 BLEU:54.05 SacreBLEU:54.31 ROUGE1:76.13 ROUGE2:57.22 ROUGEL:74.90 ROUGELsum:74.89
count stop: 5


In [ ]:
! cp '/content/indonlg/save/id-jv/example/best_model_0.th' '/content/drive/My Drive/best_model_multi_id_jv'

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/indonlg/save/id-jv/example/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 150/150 [04:09<00:00,  1.66s/it]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_id-jv.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_id-jv.csv")

== Prediction Result ==
                                                 hyp  \
0  pada waktu itu her odes, raja wilayah, mendeng...   
1  " dan dalam keadaan demikian, ketika aku denga...   
2  karena itu ia tahu, sebelum dan mengatakan ten...   
3                      kasih karunia menyertai kamu.   
4  dengan tanganku sendiri aku mengerjakan pekerj...   

                                               label  
0  pada masa itu sampailah berita - berita tentan...  
1  " dan dalam keadaan demikian, ketika aku denga...  
2  karena itu ia telah melihat ke depan dan telah...  
3             kasih karunia menyertai kamu sekalian.  
4  kami melakukan pekerjaan tangan yang berat. ka...  

== Model Performance ==
            BLEU  SacreBLEU    ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.000000   1.000000   1.00000   1.000000   1.000000   1.000000
mean   44.352344  44.604752  66.42391  47.522311  63.776134  63.825427
std          NaN        NaN       NaN        NaN        NaN        NaN


## id-su



In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = True
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 8
valid_batch_size = 8
test_batch_size = 8

source_lang = "[indonesian]"
target_lang = "[sundanese]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/id-su/example'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:

train_loader = GenerationDataLoader(dataset=train_dataset_su, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_su, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_su, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:0.1660 LR:0.00010000: 100%|██████████| 746/746 [04:32<00:00,  2.74it/s]


(Epoch 1) TRAIN LOSS:0.1660 BLEU:90.26 SacreBLEU:90.59 ROUGE1:95.60 ROUGE2:91.08 ROUGEL:95.53 ROUGELsum:95.53 LR:0.00010000


VALID LOSS:2.0859: 100%|██████████| 100/100 [00:12<00:00,  8.22it/s]


(Epoch 1) VALID LOSS:2.0859 BLEU:36.06 SacreBLEU:36.40 ROUGE1:62.70 ROUGE2:38.38 ROUGEL:60.04 ROUGELsum:60.04


(Epoch 2) TRAIN LOSS:0.1484 LR:0.00009000: 100%|██████████| 746/746 [04:32<00:00,  2.74it/s]


...

(Epoch 6) VALID LOSS:2.2911 BLEU:36.18 SacreBLEU:36.52 ROUGE1:62.83 ROUGE2:38.33 ROUGEL:60.03 ROUGELsum:60.04
count stop: 4


(Epoch 7) TRAIN LOSS:0.0327 LR:0.00005314: 100%|██████████| 746/746 [04:31<00:00,  2.75it/s]


(Epoch 7) TRAIN LOSS:0.0327 BLEU:98.29 SacreBLEU:98.34 ROUGE1:99.29 ROUGE2:98.51 ROUGEL:99.29 ROUGELsum:99.29 LR:0.00005314


VALID LOSS:2.3224: 100%|██████████| 100/100 [00:11<00:00,  8.34it/s]


(Epoch 7) VALID LOSS:2.3224 BLEU:36.02 SacreBLEU:36.39 ROUGE1:62.62 ROUGE2:38.17 ROUGEL:59.90 ROUGELsum:59.84
count stop: 5


In [ ]:
! cp '/content/indonlg/save/su-id/example/best_model_0.th' '/content/drive/My Drive/best_model_multi_id_su'

cp: cannot stat '/content/indonlg/save/su-id/example/best_model_0.th': No such file or directory


In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/indonlg/save/id-su/example/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 150/150 [03:51<00:00,  1.55s/it]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_id-su.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_id-su.csv")

== Prediction Result ==
                                                 hyp  \
0  aku tidak tahu apa yang engkau maksud, dan apa...   
1          dan mereka semuanya makan sampai kenyang.   
2  sebab kita adalah buatan allah, yang telah dic...   
3  bagi orang - orang yang kudus segala sesuatu a...   
4  dan setelah mereka membebaskan petrus dan yoha...   

                                               label  
0  sebab memang aku tidak sadar akan sesuatu, tet...  
1          dan mereka semuanya makan sampai kenyang.  
2  karena kita ini buatan allah, diciptakan dalam...  
3  bagi orang suci semuanya suci ; tetapi bagi or...  
4  sesudah dilepaskan pergilah petrus dan yohanes...  

== Model Performance ==
           BLEU  SacreBLEU     ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.00000   1.000000   1.000000   1.000000   1.000000   1.000000
mean   23.36826  23.682211  46.353319  25.295203  41.609348  41.626009
std         NaN        NaN        NaN        NaN        NaN        NaN


## id-min

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = True
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 8
valid_batch_size = 8
test_batch_size = 8

source_lang = "[indonesian]"
target_lang = "[indonesian]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/min-id/example'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:
train_dataset_min_path = '/content/drive/MyDrive/11-737/FinalProject/data/min-id/train_preprocess.json'
valid_dataset_min_path = '/content/drive/MyDrive/11-737/FinalProject/data/min-id/valid_preprocess.json'
test_dataset_min_path = '/content/drive/MyDrive/11-737/FinalProject/data/min-id/test_preprocess.json'

train_dataset_min = MachineTranslationDataset(train_dataset_min_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset_min = MachineTranslationDataset(valid_dataset_min_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset_min = MachineTranslationDataset(test_dataset_min_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

In [ ]:

train_loader = GenerationDataLoader(dataset=train_dataset_min, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_min, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_min, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:1.2477 LR:0.00010000: 100%|██████████| 1447/1447 [09:40<00:00,  2.49it/s]


(Epoch 1) TRAIN LOSS:1.2477 BLEU:66.43 SacreBLEU:67.16 ROUGE1:79.58 ROUGE2:65.75 ROUGEL:78.92 ROUGELsum:78.92 LR:0.00010000


VALID LOSS:0.9772: 100%|██████████| 200/200 [00:25<00:00,  7.79it/s]


(Epoch 1) VALID LOSS:0.9772 BLEU:68.62 SacreBLEU:68.64 ROUGE1:83.27 ROUGE2:71.03 ROUGEL:82.79 ROUGELsum:82.79


(Epoch 2) TRAIN LOSS:0.6204 LR:0.00009000: 100%|██████████| 1447/1447 [09:43<00:00,  2.48it/s]


...

(Epoch 9) VALID LOSS:1.1528 BLEU:72.70 SacreBLEU:72.72 ROUGE1:85.38 ROUGE2:74.53 ROUGEL:85.00 ROUGELsum:84.99


(Epoch 10) TRAIN LOSS:0.0291 LR:0.00003874: 100%|██████████| 1447/1447 [09:39<00:00,  2.49it/s]


(Epoch 10) TRAIN LOSS:0.0291 BLEU:98.51 SacreBLEU:98.54 ROUGE1:99.36 ROUGE2:98.72 ROUGEL:99.36 ROUGELsum:99.36 LR:0.00003874


VALID LOSS:1.1392: 100%|██████████| 200/200 [00:25<00:00,  7.72it/s]


(Epoch 10) VALID LOSS:1.1392 BLEU:72.58 SacreBLEU:72.59 ROUGE1:85.36 ROUGE2:74.55 ROUGEL:85.00 ROUGELsum:84.99
count stop: 1


In [ ]:
! cp '/content/indonlg/save/min-id/example/best_model_0.th' '/content/drive/My Drive/best_model_multi_id_min'

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/indonlg/save/min-id/example/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 400/400 [15:15<00:00,  2.29s/it]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_id-min.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_id-min.csv")

== Prediction Result ==
                                                 hyp  \
0  kehadiran & qu ot ; bo eka mata & qu ; menyusu...   
1  sop hr onica o ch re ic eps ad olah kumbang ta...   
2  umum n yo manusia jo mudah man ari mo prinsip ...   
3  bagian kode nan mang ara jo an tugas  <0x40> -...   
4  namun panik ahan tasa bu ik ind ak bar akh ia ...   

                                               label  
0  kah ad iran & qu ot ; bo eka mata & qu ot ; ma...  
1  sop hr onica o ch re ic eps ad olah kumbang ta...  
2  umum n yo manusia jo mudah man ari mo prinsip ...  
3  bagian kode nan mang ara jo an tugas  <0x40> -...  
4  namun panik ahan tu ind ak bar akh ia elok , s...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.000000   1.000000   1.000000   1.000000   1.000000   1.000000
mean   64.136503  64.148338  79.837683  69.390179  79.248965  79.263537
std          NaN        NaN        NaN        NaN        NaN        

# FLORES data

## Test FLORES jv-id data

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = False
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 8
valid_batch_size = 8
test_batch_size = 8

source_lang = "[javanese]"
target_lang = "[indonesian]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/jav-id/example'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:
test_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/test_preprocess.json'

test_dataset_flores = MachineTranslationDataset(test_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

test_loader = GenerationDataLoader(dataset=test_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/11-737/FinalProject/models/best_model_multi_jv_id'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 127/127 [01:31<00:00,  1.39it/s]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_flores_jv-id.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_flores_jv-id.csv")

== Prediction Result ==
                                                 hyp  \
0  " sek arang kita mempunyai umur - sur at yang ...   
1  dr. eh ud ur, profesor ilmu kedokteran dalam u...   
2  dan beberapa ahli lain, ia tidak yakin apa yan...   
3  pada hari senin, sara us, seorang anggota teta...   
4  kata dani us : " sek arang kita tidak melakuka...   

                                               label  
0  “ sa at ini ada menc it umur 4 bulan n ond ia ...  
1  dr. eh ud ur, profesor kedokteran di dal ho us...  
2  seperti halnya ahli - ah li lain, dia merasa s...  
3  sara dani us, sekretaris permanen komite nobel...  
4  " dani us berkata, " sa at ini kita tidak mela...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1    ROUGE2     ROUGEL  ROUGELsum
count   1.000000    1.00000   1.000000   1.00000   1.000000   1.000000
mean   12.881332   13.07301  36.565308  15.00449  34.379372  34.407054
std          NaN        NaN        NaN       NaN        NaN        NaN


In [ ]:
! cp '/content/indonlg/save/jav-id/example/prediction_result_flores_jv-id.csv' '/content/drive/My Drive/prediction_result_flores_jv-id.csv'

## Test FLORES id-jv data

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = True
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 8
valid_batch_size = 8
test_batch_size = 8

source_lang = "[javanese]"
target_lang = "[indonesian]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/id-jv/example'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:

test_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/test_preprocess.json'

test_dataset_flores = MachineTranslationDataset(test_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

test_loader = GenerationDataLoader(dataset=test_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/11-737/FinalProject/models/best_model_multi_id_jv'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 127/127 [01:26<00:00,  1.47it/s]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_flores_id-jv.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_flores_id-jv.csv")

== Prediction Result ==
                                                 hyp  \
0  " orang ini ada kira - k ira umur 4 bulan n on...   
1  dr. hai ud ur, profesor kedokteran di dal ho u...   
2  seperti halnya ahli - ah li lain ia merasa bag...   
3  sara dani us, sekretaris permanen komite nobel...   
4  " sa at ini kita tidak melakukan apa - apa. sa...   

                                               label  
0  " sa iki kita nduweni tikus umur - 4 - s asi s...  
1  dr. eh ud ur, profesor ilmu kedokteran ing uni...  
2  kaya sawetara ahli liyane, dheweke ora yakin a...  
3  ing dina senin, sara dani us, sekretaris tetep...  
4  dani us ngomong, “ sa iki kita ora nindakake a...  

== Model Performance ==
           BLEU  SacreBLEU     ROUGE1    ROUGE2     ROUGEL  ROUGELsum
count  1.000000   1.000000   1.000000  1.000000   1.000000    1.00000
mean   7.243602   7.365961  20.660364  7.210839  20.202659   20.17626
std         NaN        NaN        NaN       NaN        NaN        NaN
min 

In [ ]:
! cp '/content/indonlg/save/id-jv/example/prediction_result_flores_id-jv.csv' '/content/drive/My Drive/prediction_result_flores_id-jv.csv'

## Fine-tune on FLORES WikiMatrix data jv-id (20k)

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = False
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 4
valid_batch_size = 8
test_batch_size = 8

source_lang = "[javanese]"
target_lang = "[indonesian]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/jv-id/example'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:
train_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/train_preprocess.json'
valid_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/valid_preprocess.json'
test_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/test_preprocess.json'

train_dataset_flores = MachineTranslationDataset(train_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset_flores = MachineTranslationDataset(valid_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset_flores = MachineTranslationDataset(test_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_loader = GenerationDataLoader(dataset=train_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/drive/MyDrive/11-737/FinalProject/models/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:2.1796 LR:0.00010000: 100%|██████████| 4899/4899 [11:23<00:00,  7.17it/s]


(Epoch 1) TRAIN LOSS:2.1796 BLEU:59.84 SacreBLEU:61.21 ROUGE1:68.50 ROUGE2:52.32 ROUGEL:67.74 ROUGELsum:67.75 LR:0.00010000


VALID LOSS:3.6706: 100%|██████████| 125/125 [00:05<00:00, 21.72it/s]


(Epoch 1) VALID LOSS:3.6706 BLEU:21.21 SacreBLEU:21.43 ROUGE1:46.09 ROUGE2:22.06 ROUGEL:44.09 ROUGELsum:44.07


(Epoch 2) TRAIN LOSS:1.4294 LR:0.00009000: 100%|██████████| 4899/4899 [12:18<00:00,  6.63it/s]


...

(Epoch 7) VALID LOSS:4.6017 BLEU:21.26 SacreBLEU:21.45 ROUGE1:44.79 ROUGE2:21.79 ROUGEL:42.87 ROUGELsum:42.87
count stop: 4


(Epoch 8) TRAIN LOSS:0.1031 LR:0.00004783: 100%|██████████| 4899/4899 [11:23<00:00,  7.17it/s]


(Epoch 8) TRAIN LOSS:0.1031 BLEU:94.14 SacreBLEU:94.33 ROUGE1:97.88 ROUGE2:95.72 ROUGEL:97.84 ROUGELsum:97.84 LR:0.00004783


VALID LOSS:4.7186: 100%|██████████| 125/125 [00:05<00:00, 21.31it/s]


(Epoch 8) VALID LOSS:4.7186 BLEU:21.15 SacreBLEU:21.33 ROUGE1:44.89 ROUGE2:22.06 ROUGEL:42.94 ROUGELsum:42.90
count stop: 5


In [ ]:
! cp '/content/indonlg/save/jv-id/example/best_model_0.th' '/content/drive/My Drive/best_model_multi_jv_id_flores'

In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_multi_jv_id_flores'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 127/127 [02:33<00:00,  1.21s/it]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_flores_jv-id.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_flores_jv-id.csv")

== Prediction Result ==
                                                 hyp  \
0  " sek arang kita memiliki tikus berumur - 4 - ...   
1  dr. eh ud ur, profesor ilmu kedokteran di univ...   
2  di antara mereka, ia tidak meyakini bahwa diab...   
3  pada hari senin, sara dani us, sekretaris teta...   
4  dani us menjawab, " sa ini kita tidak melakuka...   

                                               label  
0  “ sa at ini ada menc it umur 4 bulan n ond ia ...  
1  dr. eh ud ur, profesor kedokteran di dal ho us...  
2  seperti halnya ahli - ah li lain, dia merasa s...  
3  sara dani us, sekretaris permanen komite nobel...  
4  " dani us berkata, " sa at ini kita tidak mela...  

== Model Performance ==
            BLEU  SacreBLEU    ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.000000   1.000000   1.00000   1.000000   1.000000   1.000000
mean   23.832346  24.078198  49.45339  26.277222  47.451796  47.408677
std          NaN        NaN       NaN        NaN        NaN        NaN


In [ ]:
! cp '/content/indonlg/save/jv-id/example/prediction_result_flores_jv-id.csv' '/content/drive/My Drive/prediction_result_flores_jv-id_ft.csv'

## Fine-tune on FLORES WikiMatrix data id-jv (20k)

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = True
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 4
valid_batch_size = 8
test_batch_size = 8

source_lang = "[indonesian]"
target_lang = "[javanese]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/id-jv/example'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:
train_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/train_preprocess.json'
valid_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/valid_preprocess.json'
test_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/test_preprocess.json'

train_dataset_flores = MachineTranslationDataset(train_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset_flores = MachineTranslationDataset(valid_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset_flores = MachineTranslationDataset(test_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_loader = GenerationDataLoader(dataset=train_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
torch.cuda.empty_cache() 

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/drive/MyDrive/11-737/FinalProject/models/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:2.2689 LR:0.00010000: 100%|██████████| 4899/4899 [11:27<00:00,  7.12it/s]


(Epoch 1) TRAIN LOSS:2.2689 BLEU:56.05 SacreBLEU:57.13 ROUGE1:65.21 ROUGE2:47.15 ROUGEL:64.09 ROUGELsum:64.09 LR:0.00010000


VALID LOSS:4.5036: 100%|██████████| 125/125 [00:05<00:00, 21.54it/s]


(Epoch 1) VALID LOSS:4.5036 BLEU:13.87 SacreBLEU:13.76 ROUGE1:35.64 ROUGE2:14.50 ROUGEL:34.14 ROUGELsum:34.14


(Epoch 2) TRAIN LOSS:1.4607 LR:0.00009000: 100%|██████████| 4899/4899 [11:32<00:00,  7.08it/s]


...

(Epoch 7) VALID LOSS:5.7195 BLEU:14.24 SacreBLEU:14.22 ROUGE1:35.59 ROUGE2:14.83 ROUGEL:34.02 ROUGELsum:33.96
count stop: 4


(Epoch 8) TRAIN LOSS:0.1156 LR:0.00004783: 100%|██████████| 4899/4899 [11:19<00:00,  7.21it/s]


(Epoch 8) TRAIN LOSS:0.1156 BLEU:93.67 SacreBLEU:93.85 ROUGE1:97.27 ROUGE2:94.50 ROUGEL:97.23 ROUGELsum:97.23 LR:0.00004783


VALID LOSS:5.8626: 100%|██████████| 125/125 [00:05<00:00, 21.43it/s]


(Epoch 8) VALID LOSS:5.8626 BLEU:14.27 SacreBLEU:14.25 ROUGE1:35.64 ROUGE2:14.87 ROUGEL:34.06 ROUGELsum:34.04
count stop: 5


In [ ]:
! cp '/content/indonlg/save/id-jv/example/best_model_0.th' '/content/drive/My Drive/best_model_multi_id_jv_flores'

In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_multi_id_jv_flores'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 127/127 [02:00<00:00,  1.05it/s]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_flores_id-jv_20k.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_flores_id-jv.csv")

== Prediction Result ==
                                                 hyp  \
0  “ sa at iki ana ing umur 4 sasi n ond ia b ete...   
1  dr. eh ud ur, profesor ked h okter an ing dal ...   
2  kaya halnya ahli - ah li liyané, panjenengané ...   
3  sara dani us, sekretaris permanen komite nobel...   
4  " dani us ngomong, " sa at iki kita ora melaku...   

                                               label  
0  " sa iki kita nduweni tikus umur - 4 - s asi s...  
1  dr. eh ud ur, profesor ilmu kedokteran ing uni...  
2  kaya sawetara ahli liyane, dheweke ora yakin a...  
3  ing dina senin, sara dani us, sekretaris tetep...  
4  dani us ngomong, “ sa iki kita ora nindakake a...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.000000   1.000000   1.000000   1.000000   1.000000   1.000000
mean   12.227321  12.359807  32.932222  15.068199  31.654913  31.627528
std          NaN        NaN        NaN        NaN        NaN        

In [ ]:
! cp '/content/indonlg/save/id-jv/example/prediction_result_flores_id-jv_20k.csv' '/content/drive/My Drive/prediction_result_flores_id-jv_ft.csv'

## **Ablation** with FLORES data

### jv-id 5k

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = False
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 4
valid_batch_size = 8
test_batch_size = 8

source_lang = "[javanese]"
target_lang = "[indonesian]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/jv-id/5k'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:
train_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/train_preprocess_5k.json'
valid_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/valid_preprocess.json'
test_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/test_preprocess.json'

train_dataset_flores = MachineTranslationDataset(train_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset_flores = MachineTranslationDataset(valid_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset_flores = MachineTranslationDataset(test_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_loader = GenerationDataLoader(dataset=train_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/drive/MyDrive/11-737/FinalProject/models/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

Token indices sequence length is longer than the specified maximum sequence length for this model (1825 > 1024). Running this sequence through the model will result in indexing errors
(Epoch 1) TRAIN LOSS:1.7910 LR:0.00010000: 100%|██████████| 1250/1250 [02:49<00:00,  7.36it/s]


(Epoch 1) TRAIN LOSS:1.7910 BLEU:62.28 SacreBLEU:63.68 ROUGE1:74.46 ROUGE2:58.18 ROUGEL:74.04 ROUGELsum:74.03 LR:0.00010000


VALID LOSS:3.9425: 100%|██████████| 125/125 [00:05<00:00, 21.91it/s]


(Epoch 1) VALID LOSS:3.9425 BLEU:19.85 SacreBLEU:20.08 ROUGE1:44.68 ROUGE2:21.05 ROUGEL:42.84 ROUGELsum:42.86


(Epoch 2) TRAIN LOSS:0.8539 LR:0.00009000: 100%|██████████| 1250/1250 [02:48<00:00,  7.41it/s]


...

(Epoch 5) VALID LOSS:4.8768 BLEU:19.47 SacreBLEU:19.69 ROUGE1:43.67 ROUGE2:20.38 ROUGEL:41.86 ROUGELsum:41.89
count stop: 4


(Epoch 6) TRAIN LOSS:0.0836 LR:0.00005905: 100%|██████████| 1250/1250 [02:29<00:00,  8.37it/s]


(Epoch 6) TRAIN LOSS:0.0836 BLEU:95.27 SacreBLEU:95.44 ROUGE1:98.54 ROUGE2:97.08 ROUGEL:98.52 ROUGELsum:98.53 LR:0.00005905


VALID LOSS:5.0342: 100%|██████████| 125/125 [00:05<00:00, 21.90it/s]


(Epoch 6) VALID LOSS:5.0342 BLEU:19.63 SacreBLEU:19.87 ROUGE1:44.15 ROUGE2:20.88 ROUGEL:42.29 ROUGELsum:42.32
count stop: 5


In [ ]:
! cp '/content/indonlg/save/jv-id/5k/best_model_0.th' '/content/drive/My Drive/best_model_multi_jv_id_flores_5k.th'

In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_multi_jv_id_flores_5k.th'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 127/127 [01:36<00:00,  1.32it/s]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_flores_jv-id_5k.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_flores_jv-id_5k.csv")

== Prediction Result ==
                                                 hyp  \
0  " sa iki kita mempunyai tikus umur - 4 - s asi...   
1  dr. eh ud ur, profesor ilmu kedokteran di univ...   
2  seperti seorang ahli lainnya, ia tidak yakin b...   
3  pada hari senin, sara dani us, sekretaris teta...   
4  dani us berkata : " sa iki kita tidak meminta ...   

                                               label  
0  “ sa at ini ada menc it umur 4 bulan n ond ia ...  
1  dr. eh ud ur, profesor kedokteran di dal ho us...  
2  seperti halnya ahli - ah li lain, dia merasa s...  
3  sara dani us, sekretaris permanen komite nobel...  
4  " dani us berkata, " sa at ini kita tidak mela...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1    ROUGE2     ROUGEL  ROUGELsum
count   1.000000   1.000000   1.000000   1.00000   1.000000   1.000000
mean   21.778526  21.975736  47.682179  24.27871  45.712139  45.725216
std          NaN        NaN        NaN       NaN        NaN        NaN


In [ ]:
! cp '/content/indonlg/save/jv-id/5k/prediction_result_flores_jv-id_5k.csv' '/content/drive/My Drive/prediction_result_flores_jv-id_5k.csv'

### jv-id 10k

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = False
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 4
valid_batch_size = 8
test_batch_size = 8

source_lang = "[javanese]"
target_lang = "[indonesian]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/jv-id/10k'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:
train_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/train_preprocess_10k.json'
valid_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/valid_preprocess.json'
test_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/test_preprocess.json'

train_dataset_flores = MachineTranslationDataset(train_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset_flores = MachineTranslationDataset(valid_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset_flores = MachineTranslationDataset(test_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_loader = GenerationDataLoader(dataset=train_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/drive/MyDrive/11-737/FinalProject/models/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:1.8335 LR:0.00010000: 100%|██████████| 2500/2500 [05:09<00:00,  8.06it/s]


(Epoch 1) TRAIN LOSS:1.8335 BLEU:62.71 SacreBLEU:64.13 ROUGE1:73.53 ROUGE2:57.55 ROUGEL:73.01 ROUGELsum:73.00 LR:0.00010000


VALID LOSS:3.7577: 100%|██████████| 125/125 [00:05<00:00, 21.89it/s]


(Epoch 1) VALID LOSS:3.7577 BLEU:20.84 SacreBLEU:21.10 ROUGE1:45.86 ROUGE2:21.96 ROUGEL:43.96 ROUGELsum:44.00


(Epoch 2) TRAIN LOSS:1.0105 LR:0.00009000: 100%|██████████| 2500/2500 [05:06<00:00,  8.15it/s]


...

(Epoch 6) VALID LOSS:4.8227 BLEU:20.70 SacreBLEU:20.93 ROUGE1:44.86 ROUGE2:21.77 ROUGEL:43.02 ROUGELsum:43.01
count stop: 4


(Epoch 7) TRAIN LOSS:0.0803 LR:0.00005314: 100%|██████████| 2500/2500 [05:03<00:00,  8.23it/s]


(Epoch 7) TRAIN LOSS:0.0803 BLEU:95.41 SacreBLEU:95.56 ROUGE1:98.40 ROUGE2:96.78 ROUGEL:98.39 ROUGELsum:98.39 LR:0.00005314


VALID LOSS:4.9038: 100%|██████████| 125/125 [00:05<00:00, 22.22it/s]


(Epoch 7) VALID LOSS:4.9038 BLEU:20.71 SacreBLEU:20.92 ROUGE1:45.02 ROUGE2:21.64 ROUGEL:43.18 ROUGELsum:43.22
count stop: 5


In [ ]:
! cp '/content/indonlg/save/jv-id/10k/best_model_0.th' '/content/drive/My Drive/best_model_multi_jv_id_flores_10k.th'

In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_multi_jv_id_flores_10k.th'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 127/127 [01:37<00:00,  1.30it/s]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_flores_jv-id_10k.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_flores_jv-id_10k.csv")

== Prediction Result ==
                                                 hyp  \
0  " k ini kita memiliki tikus berusia - 4 - s as...   
1  dr. eh ud ur, profesor ilmu kedokteran di univ...   
2  seperti beberapa ahli lainnya, ia tidak yakin ...   
3  pada hari senin, sara dani us, sekretaris teta...   
4  dani us ngomong, “ sa iki kita tidak melakukan...   

                                               label  
0  “ sa at ini ada menc it umur 4 bulan n ond ia ...  
1  dr. eh ud ur, profesor kedokteran di dal ho us...  
2  seperti halnya ahli - ah li lain, dia merasa s...  
3  sara dani us, sekretaris permanen komite nobel...  
4  " dani us berkata, " sa at ini kita tidak mela...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.000000   1.000000   1.000000   1.000000   1.000000   1.000000
mean   23.052617  23.255248  48.497308  25.351908  46.496824  46.501106
std          NaN        NaN        NaN        NaN        NaN        

In [ ]:
! cp '/content/indonlg/save/jv-id/10k/prediction_result_flores_jv-id_10k.csv' '/content/drive/My Drive/prediction_result_flores_jv-id_10k.csv'

### jv-id 15k

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = False
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 4
valid_batch_size = 8
test_batch_size = 8

source_lang = "[javanese]"
target_lang = "[indonesian]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/jv-id/15k'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:
train_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/train_preprocess_15k.json'
valid_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/valid_preprocess.json'
test_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/test_preprocess.json'

train_dataset_flores = MachineTranslationDataset(train_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset_flores = MachineTranslationDataset(valid_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset_flores = MachineTranslationDataset(test_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_loader = GenerationDataLoader(dataset=train_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/drive/MyDrive/11-737/FinalProject/models/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:1.9835 LR:0.00010000: 100%|██████████| 3750/3750 [07:51<00:00,  7.95it/s]


(Epoch 1) TRAIN LOSS:1.9835 BLEU:61.63 SacreBLEU:62.92 ROUGE1:71.35 ROUGE2:55.30 ROUGEL:70.72 ROUGELsum:70.71 LR:0.00010000


VALID LOSS:3.7817: 100%|██████████| 125/125 [00:05<00:00, 22.35it/s]


(Epoch 1) VALID LOSS:3.7817 BLEU:21.39 SacreBLEU:21.62 ROUGE1:46.22 ROUGE2:22.42 ROUGEL:44.30 ROUGELsum:44.37


(Epoch 2) TRAIN LOSS:1.2048 LR:0.00009000: 100%|██████████| 3750/3750 [07:48<00:00,  8.01it/s]


...

(Epoch 9) VALID LOSS:4.9231 BLEU:21.27 SacreBLEU:21.47 ROUGE1:45.48 ROUGE2:22.23 ROUGEL:43.59 ROUGELsum:43.55
count stop: 4


(Epoch 10) TRAIN LOSS:0.0460 LR:0.00003874: 100%|██████████| 3750/3750 [07:57<00:00,  7.85it/s]


(Epoch 10) TRAIN LOSS:0.0460 BLEU:97.05 SacreBLEU:97.14 ROUGE1:99.18 ROUGE2:98.31 ROUGEL:99.17 ROUGELsum:99.17 LR:0.00003874


VALID LOSS:5.0615: 100%|██████████| 125/125 [00:05<00:00, 21.13it/s]


(Epoch 10) VALID LOSS:5.0615 BLEU:21.43 SacreBLEU:21.62 ROUGE1:45.70 ROUGE2:22.43 ROUGEL:43.76 ROUGELsum:43.71
count stop: 5


In [ ]:
! cp '/content/indonlg/save/jv-id/15k/best_model_0.th' '/content/drive/My Drive/best_model_multi_jv_id_flores_15k.th'

In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_multi_jv_id_flores_15k.th'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 127/127 [01:39<00:00,  1.28it/s]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_flores_jv-id_15k.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_flores_jv-id_15k.csv")

== Prediction Result ==
                                                 hyp  \
0  " sek arang kita memiliki umur - 4 - s asi yan...   
1  dr. eh ud ur, profesor ilmu kedokteran di univ...   
2  seperti beberapa ahli lainnya, ia tak yakin ba...   
3  pada hari senin, sara dani us, sekretaris teta...   
4  dani us berkata, “ sek arang kita tidak melaku...   

                                               label  
0  “ sa at ini ada menc it umur 4 bulan n ond ia ...  
1  dr. eh ud ur, profesor kedokteran di dal ho us...  
2  seperti halnya ahli - ah li lain, dia merasa s...  
3  sara dani us, sekretaris permanen komite nobel...  
4  " dani us berkata, " sa at ini kita tidak mela...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.000000   1.000000   1.000000   1.000000   1.000000   1.000000
mean   23.323154  23.545992  49.090198  25.594668  47.065063  47.001526
std          NaN        NaN        NaN        NaN        NaN        

In [ ]:
! cp '/content/indonlg/save/jv-id/15k/prediction_result_flores_jv-id_15k.csv' '/content/drive/My Drive/prediction_result_flores_jv-id_15k.csv'

### id-jv 5k

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = True
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 4
valid_batch_size = 8
test_batch_size = 8

source_lang = "[indonesian]"
target_lang = "[javanese]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/id-jv/5k'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:
train_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/train_preprocess_5k.json'
valid_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/valid_preprocess.json'
test_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/test_preprocess.json'

train_dataset_flores = MachineTranslationDataset(train_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset_flores = MachineTranslationDataset(valid_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset_flores = MachineTranslationDataset(test_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_loader = GenerationDataLoader(dataset=train_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/drive/MyDrive/11-737/FinalProject/models/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:2.2742 LR:0.00010000: 100%|██████████| 1250/1250 [02:37<00:00,  7.93it/s]


(Epoch 1) TRAIN LOSS:2.2742 BLEU:54.58 SacreBLEU:55.95 ROUGE1:66.88 ROUGE2:48.73 ROUGEL:66.13 ROUGELsum:66.12 LR:0.00010000


VALID LOSS:5.0470: 100%|██████████| 125/125 [00:06<00:00, 20.47it/s]


(Epoch 1) VALID LOSS:5.0470 BLEU:11.51 SacreBLEU:11.49 ROUGE1:32.38 ROUGE2:12.59 ROUGEL:30.88 ROUGELsum:30.87


Token indices sequence length is longer than the specified maximum sequence length for this model (1779 > 1024). Running this sequence through the model will result in indexing errors
(Epoch 2) TRAIN LOSS:1.1755 LR:0.00009000: 100%|██████████| 1250/1250 [02:34<00:00,  8.07it/s]


...

(Epoch 7) VALID LOSS:6.4151 BLEU:12.63 SacreBLEU:12.63 ROUGE1:33.24 ROUGE2:13.43 ROUGEL:31.82 ROUGELsum:31.83
count stop: 4


(Epoch 8) TRAIN LOSS:0.0600 LR:0.00004783: 100%|██████████| 1250/1250 [02:35<00:00,  8.05it/s]


(Epoch 8) TRAIN LOSS:0.0600 BLEU:96.86 SacreBLEU:96.95 ROUGE1:99.04 ROUGE2:98.05 ROUGEL:99.03 ROUGELsum:99.03 LR:0.00004783


VALID LOSS:6.5137: 100%|██████████| 125/125 [00:06<00:00, 20.16it/s]


(Epoch 8) VALID LOSS:6.5137 BLEU:12.64 SacreBLEU:12.65 ROUGE1:33.30 ROUGE2:13.38 ROUGEL:31.94 ROUGELsum:31.91
count stop: 5


In [ ]:
! cp '/content/indonlg/save/id-jv/5k/best_model_0.th' '/content/drive/My Drive/best_model_multi_id_jv_flores_5k.th'

In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_multi_id_jv_flores_5k.th'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 127/127 [01:53<00:00,  1.12it/s]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_flores_id-jv_5k.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_flores_id-jv_5k.csv")

== Prediction Result ==
                                                 hyp  \
0  “ sa at punika wonten ing umur 4 wulan n ond i...   
1  dr. eh ud ur, profesor kedokteran ing dal ho u...   
2  kados hal ipun ahli - ah li san <0xC3> <0xA8> ...   
3  sara dani us, sekretaris permanen komite nobel...   
4  " dani us berkata, " sa at punika kita tidak m...   

                                               label  
0  " sa iki kita nduweni tikus umur - 4 - s asi s...  
1  dr. eh ud ur, profesor ilmu kedokteran ing uni...  
2  kaya sawetara ahli liyane, dheweke ora yakin a...  
3  ing dina senin, sara dani us, sekretaris tetep...  
4  dani us ngomong, “ sa iki kita ora nindakake a...  

== Model Performance ==
           BLEU  SacreBLEU     ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.00000   1.000000   1.000000   1.000000   1.000000   1.000000
mean   12.27524  12.426352  32.126764  14.117545  30.885382  30.842333
std         NaN        NaN        NaN        NaN        NaN        NaN


In [ ]:
! cp '/content/indonlg/save/id-jv/5k/prediction_result_flores_id-jv_5k.csv' '/content/drive/My Drive/prediction_result_flores_id-jv_5k.csv'

### id-jv 10k

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = True
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 4
valid_batch_size = 8
test_batch_size = 8

source_lang = "[indonesian]"
target_lang = "[javanese]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/id-jv/10k'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:
train_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/train_preprocess_10k.json'
valid_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/valid_preprocess.json'
test_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/test_preprocess.json'

train_dataset_flores = MachineTranslationDataset(train_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset_flores = MachineTranslationDataset(valid_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset_flores = MachineTranslationDataset(test_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_loader = GenerationDataLoader(dataset=train_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/drive/MyDrive/11-737/FinalProject/models/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:2.1646 LR:0.00010000: 100%|██████████| 2500/2500 [05:17<00:00,  7.86it/s]


(Epoch 1) TRAIN LOSS:2.1646 BLEU:56.26 SacreBLEU:57.50 ROUGE1:67.57 ROUGE2:49.65 ROUGEL:66.68 ROUGELsum:66.67 LR:0.00010000


VALID LOSS:4.7764: 100%|██████████| 125/125 [00:06<00:00, 20.50it/s]


(Epoch 1) VALID LOSS:4.7764 BLEU:13.87 SacreBLEU:13.82 ROUGE1:35.94 ROUGE2:14.67 ROUGEL:34.12 ROUGELsum:34.15


(Epoch 2) TRAIN LOSS:1.2286 LR:0.00009000: 100%|██████████| 2500/2500 [05:14<00:00,  7.96it/s]


...

(Epoch 7) VALID LOSS:6.2762 BLEU:13.48 SacreBLEU:13.53 ROUGE1:34.53 ROUGE2:14.11 ROUGEL:32.83 ROUGELsum:32.87
count stop: 4


(Epoch 8) TRAIN LOSS:0.0734 LR:0.00004783: 100%|██████████| 2500/2500 [05:21<00:00,  7.78it/s]


(Epoch 8) TRAIN LOSS:0.0734 BLEU:96.02 SacreBLEU:96.15 ROUGE1:98.51 ROUGE2:96.98 ROUGEL:98.50 ROUGELsum:98.50 LR:0.00004783


VALID LOSS:6.2790: 100%|██████████| 125/125 [00:06<00:00, 19.87it/s]


(Epoch 8) VALID LOSS:6.2790 BLEU:13.73 SacreBLEU:13.72 ROUGE1:34.88 ROUGE2:14.27 ROUGEL:33.26 ROUGELsum:33.26
count stop: 5


In [ ]:
! cp '/content/indonlg/save/id-jv/10k/best_model_0.th' '/content/drive/My Drive/best_model_multi_id_jv_flores_10k.th'

In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_multi_id_jv_flores_10k.th'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 127/127 [01:57<00:00,  1.08it/s]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_flores_id-jv_10k.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_flores_id-jv_10k.csv")

== Prediction Result ==
                                                 hyp  \
0  “ sa at iki ana kira - k ira kanggo umur 4 sas...   
1  dr. eh ud ur, profesor ked h okter an ing dal ...   
2  kaya déné e ahli - ah li san <0xC3> <0xA8> sip...   
3  sara dani us, sekretaris permanen komite nobel...   
4  " dani us berkata, " sa at punika boten nga ya...   

                                               label  
0  " sa iki kita nduweni tikus umur - 4 - s asi s...  
1  dr. eh ud ur, profesor ilmu kedokteran ing uni...  
2  kaya sawetara ahli liyane, dheweke ora yakin a...  
3  ing dina senin, sara dani us, sekretaris tetep...  
4  dani us ngomong, “ sa iki kita ora nindakake a...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.000000   1.000000   1.000000   1.000000   1.000000   1.000000
mean   12.223517  12.352098  32.965241  14.909982  31.639772  31.646339
std          NaN        NaN        NaN        NaN        NaN        

In [ ]:
! cp '/content/indonlg/save/id-jv/10k/prediction_result_flores_id-jv_10k.csv' '/content/drive/My Drive/prediction_result_flores_id-jv_10k.csv'

### id-jv 15k

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = True
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 4
valid_batch_size = 8
test_batch_size = 8

source_lang = "[indonesian]"
target_lang = "[javanese]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/id-jv/15k'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:
train_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/train_preprocess_15k.json'
valid_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/valid_preprocess.json'
test_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/test_preprocess.json'

train_dataset_flores = MachineTranslationDataset(train_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset_flores = MachineTranslationDataset(valid_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset_flores = MachineTranslationDataset(test_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_loader = GenerationDataLoader(dataset=train_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/drive/MyDrive/11-737/FinalProject/models/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:2.1828 LR:0.00010000: 100%|██████████| 3750/3750 [06:02<00:00, 10.35it/s]


(Epoch 1) TRAIN LOSS:2.1828 BLEU:56.66 SacreBLEU:57.80 ROUGE1:66.80 ROUGE2:48.95 ROUGEL:65.81 ROUGELsum:65.80 LR:0.00010000


VALID LOSS:4.5569: 100%|██████████| 125/125 [00:04<00:00, 27.94it/s]


(Epoch 1) VALID LOSS:4.5569 BLEU:14.07 SacreBLEU:13.95 ROUGE1:35.37 ROUGE2:14.56 ROUGEL:33.87 ROUGELsum:33.87


(Epoch 2) TRAIN LOSS:1.3273 LR:0.00009000: 100%|██████████| 3750/3750 [05:57<00:00, 10.50it/s]


...

(Epoch 7) VALID LOSS:5.8885 BLEU:14.11 SacreBLEU:14.08 ROUGE1:35.56 ROUGE2:14.91 ROUGEL:34.00 ROUGELsum:33.97
count stop: 4


(Epoch 8) TRAIN LOSS:0.0928 LR:0.00004783: 100%|██████████| 3750/3750 [05:56<00:00, 10.52it/s]


(Epoch 8) TRAIN LOSS:0.0928 BLEU:94.88 SacreBLEU:95.02 ROUGE1:97.97 ROUGE2:95.89 ROUGEL:97.95 ROUGELsum:97.95 LR:0.00004783


VALID LOSS:6.0121: 100%|██████████| 125/125 [00:04<00:00, 27.55it/s]


(Epoch 8) VALID LOSS:6.0121 BLEU:14.05 SacreBLEU:14.02 ROUGE1:35.44 ROUGE2:14.55 ROUGEL:33.65 ROUGELsum:33.63
count stop: 5


In [ ]:
! cp '/content/indonlg/save/id-jv/15k/best_model_0.th' '/content/drive/My Drive/best_model_multi_id_jv_flores_15k.th'

In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_multi_id_jv_flores_15k.th'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 127/127 [01:48<00:00,  1.17it/s]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_flores_id-jv_15k.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_flores_id-jv_15k.csv")

== Prediction Result ==
                                                 hyp  \
0  “ sa at iki ana ing umur 4 wulan n ond ia b et...   
1  dr. eh ud ur, prof és or ked h okter an ing da...   
2  kados ahli - ah li san <0xC3> <0xA8> sipun, pa...   
3  sara dani us, sekretaris permanen komite nobel...   
4  " dani us ngomong, " sa at iki kita ora nglak ...   

                                               label  
0  " sa iki kita nduweni tikus umur - 4 - s asi s...  
1  dr. eh ud ur, profesor ilmu kedokteran ing uni...  
2  kaya sawetara ahli liyane, dheweke ora yakin a...  
3  ing dina senin, sara dani us, sekretaris tetep...  
4  dani us ngomong, “ sa iki kita ora nindakake a...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.000000   1.000000   1.000000   1.000000   1.000000   1.000000
mean   12.300687  12.435915  32.585473  14.655203  31.315385  31.312116
std          NaN        NaN        NaN        NaN        NaN        

In [ ]:
! cp '/content/indonlg/save/id-jv/15k/prediction_result_flores_id-jv_15k.csv' '/content/drive/My Drive/prediction_result_flores_id-jv_15k.csv'

# Backtranslation

## Generate Data

In [ ]:
!pip install fastBPE
!pip install nlpaug
!pip install fairseq
!pip install sacremoses
!pip install transformers
!pip install itranslate

  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp37-cp37m-linux_x86_64.whl size=483155 sha256=24a32bfe8c2f0190faa6e1b0e6d47d2729c1f08b34eb79fdc8e7a25ac44a20d8
  Stored in directory: /root/.cache/pip/wheels/bd/d4/0e/0d317a65f77d3f8049fedd8a2ee0519164cf3e6bd77ef886f1
Successfully built fastBPE
     |████████████████████████████████| 410 kB 35.6 MB/s 
     |████████████████████████████████| 1.7 MB 32.2 MB/s 
     |████████████████████████████████| 147 kB 75.0 MB/s 
     |████████████████████████████████| 74 kB 4.5 MB/s 
     |████████████████████████████████| 112 kB 100.0 MB/s 
     |████████████████████████████████| 596 kB 73.9 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=f37b74d21bb42c181bf6ca0e5123eff5d39ae914c0e921233984b35d0bf9f73a
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime
  Attempting uninstall: P

     |████████████████████████████████| 76 kB 6.2 MB/s 
     |████████████████████████████████| 58 kB 8.2 MB/s 
     |████████████████████████████████| 54 kB 4.4 MB/s 
     |████████████████████████████████| 79 kB 10.3 MB/s 


In [ ]:
import pandas as pd
import fairseq
import fastBPE
import sacremoses
import transformers
from itranslate import itranslate
import time

import pandas as pd

In [ ]:
train_dataset_flores_path = '/content/drive/MyDrive/FinalProject/data/jv-id/flores/train_preprocess_5k.json'

In [ ]:
df = pd.read_json(train_dataset_flores_path)

In [ ]:
print(df)

        id                                               text  \
0        0  Para warga pada metu karo nggowoni sampah sing...   
1        1  Pramila putranipun ingkang asma Drestarastra-r...   
2        2  Sira banjur didangu sarupaning kabungahan papa...   
3        3              Temoné ing kana wujud prasasti watu.    
4        4         "Ivan Lanin, Bangga Berbahasa Indonésia".    
...    ...                                                ...   
4995  4995  Dat iki diasilaké kanthi cara distilasi fraksi...   
4996  4996  Robinson Crusoe ya iku buku karyanè kang pisan...   
4997  4997  Jeff Harriman (Kiefer Sutherland) lunga plesir...   
4998  4998  Ade Namnung misuwur minangka salah satunggalip...   
4999  4999  Ing taun 1850 siji Kelas Gim Olympiadé diadeka...   

                                                  label  
0     Anak-anak kemudian meninggalkan dan Norman bek...  
1     Lalu tahanlah diri dari sholat, karena sesungg...  
2     Tidakkah Allah lebih mengetahui tentang

In [ ]:
def get_augmented_data(text, src="", target="", second_hop=""):
  backtranslated_text = text
  try:
    midway = itranslate(text, to_lang=target, from_lang=src)
    second = itranslate(midway, to_lang=src, from_lang=target)
    #backtranslated_text = itranslate(second, to_lang=src, from_lang=second_hop)
  except:
    print(text, src, target)
    return "__ERROR__"
  return second

In [ ]:
!pip install pandas==1.2.5

     |████████████████████████████████| 9.9 MB 16.9 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()
df['ind'] = df.progress_apply(lambda row: get_augmented_data(row['label'], 'id', "ms"), axis=1)
df.to_csv('augmented_initial_2.csv')

[E 220430 18:32:04 itranslate:135] The read operation timed out


Magazen box adalah magazen paling populer untuk pistol dan senapan modern.  id ms


[W 220430 18:41:53 itranslate:95]  text (16819) too longer, trimmed to 5000
[E 220430 19:32:27 itranslate:135] The read operation timed out


Bangladesh: Mirip dengan Alu-tikki, tetapi diberi nama Alu-chop dan di tengahnya diisi daging sapi.  id ms


[W 220430 20:41:08 itranslate:95]  text (5353) too longer, trimmed to 5000
[W 220430 21:20:30 itranslate:95]  text (5871) too longer, trimmed to 5000


In [ ]:
print(df)

        id                                               text  \
0        0  Para warga pada metu karo nggowoni sampah sing...   
1        1  Pramila putranipun ingkang asma Drestarastra-r...   
2        2  Sira banjur didangu sarupaning kabungahan papa...   
3        3              Temoné ing kana wujud prasasti watu.    
4        4         "Ivan Lanin, Bangga Berbahasa Indonésia".    
...    ...                                                ...   
4995  4995  Dat iki diasilaké kanthi cara distilasi fraksi...   
4996  4996  Robinson Crusoe ya iku buku karyanè kang pisan...   
4997  4997  Jeff Harriman (Kiefer Sutherland) lunga plesir...   
4998  4998  Ade Namnung misuwur minangka salah satunggalip...   
4999  4999  Ing taun 1850 siji Kelas Gim Olympiadé diadeka...   

                                                  label  \
0     Anak-anak kemudian meninggalkan dan Norman bek...   
1     Lalu tahanlah diri dari sholat, karena sesungg...   
2     Tidakkah Allah lebih mengetahui tent

In [ ]:
!cp /content/augmented_initial_2.csv /content/drive/MyDrive/FinalProject

## jv-id

### With backtranslated jv

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = False
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 4
valid_batch_size = 8
test_batch_size = 8

source_lang = "[javanese]"
target_lang = "[indonesian]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/jv-id/example'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:
train_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/train_preprocess.json'
valid_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/valid_preprocess.json'
test_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/test_preprocess.json'

backtrans_jv_path = '/content/drive/MyDrive/11-737/FinalProject/data/backtrans_jav.json'

train_dataset_flores = MachineTranslationDataset(train_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset_flores = MachineTranslationDataset(valid_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset_flores = MachineTranslationDataset(test_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

backtrans_jv_dataset = MachineTranslationDataset(backtrans_jv_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_dataset = ConcatDataset([train_dataset_flores, backtrans_jv_dataset])

train_loader = GenerationDataLoader(dataset=train_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/drive/MyDrive/11-737/FinalProject/models/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:2.3800 LR:0.00010000: 100%|██████████| 6149/6149 [16:05<00:00,  6.37it/s]


(Epoch 1) TRAIN LOSS:2.3800 BLEU:56.57 SacreBLEU:58.11 ROUGE1:66.17 ROUGE2:48.61 ROUGEL:65.20 ROUGELsum:65.21 LR:0.00010000


VALID LOSS:3.4250: 100%|██████████| 125/125 [00:05<00:00, 20.89it/s]


(Epoch 1) VALID LOSS:3.4250 BLEU:21.98 SacreBLEU:22.20 ROUGE1:47.28 ROUGE2:23.14 ROUGEL:45.34 ROUGELsum:45.39


(Epoch 2) TRAIN LOSS:1.5289 LR:0.00009000: 100%|██████████| 6149/6149 [13:35<00:00,  7.54it/s]


...

(Epoch 5) VALID LOSS:4.0930 BLEU:20.09 SacreBLEU:20.29 ROUGE1:43.51 ROUGE2:20.54 ROUGEL:41.45 ROUGELsum:41.46
count stop: 4


(Epoch 6) TRAIN LOSS:0.2274 LR:0.00005905: 100%|██████████| 6149/6149 [13:31<00:00,  7.58it/s]


(Epoch 6) TRAIN LOSS:0.2274 BLEU:88.80 SacreBLEU:89.17 ROUGE1:94.91 ROUGE2:90.20 ROUGEL:94.79 ROUGELsum:94.79 LR:0.00005905


VALID LOSS:4.2787: 100%|██████████| 125/125 [00:06<00:00, 20.39it/s]


(Epoch 6) VALID LOSS:4.2787 BLEU:20.08 SacreBLEU:20.25 ROUGE1:43.62 ROUGE2:20.62 ROUGEL:41.67 ROUGELsum:41.70
count stop: 5


In [ ]:
! cp '/content/indonlg/save/jv-id/example/best_model_0.th' '/content/drive/My Drive/best_model_jv_id_backtrans_jv.th'

In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_jv_id_backtrans_jv.th'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 127/127 [01:45<00:00,  1.21it/s]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_flores_jv-id_backtrans-jv.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_flores_jv-id_backtrans-jv.csv")

== Prediction Result ==
                                                 hyp  \
0  " sa iki kita mempunyai tikus umur - 4 - s asi...   
1  dr. eh ud ur, seorang profesor ilmu kedokteran...   
2  seperti beberapa ahli lainnya, ia tidak menget...   
3  pada hari senin, sara dani us, sekretaris teta...   
4  dani us berkata : " sa iki kita tidak melakuka...   

                                               label  
0  “ sa at ini ada menc it umur 4 bulan n ond ia ...  
1  dr. eh ud ur, profesor kedokteran di dal ho us...  
2  seperti halnya ahli - ah li lain, dia merasa s...  
3  sara dani us, sekretaris permanen komite nobel...  
4  " dani us berkata, " sa at ini kita tidak mela...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.000000   1.000000   1.000000   1.000000   1.000000   1.000000
mean   23.796967  24.030851  50.128133  26.306952  48.000005  48.004119
std          NaN        NaN        NaN        NaN        NaN        

In [ ]:
! cp '/content/indonlg/save/jv-id/example/prediction_result_flores_jv-id_backtrans-jv.csv' '/content/drive/My Drive/prediction_result_flores_jv-id_backtrans-jv.csv'

### With backtranslated id

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = False
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 4
valid_batch_size = 8
test_batch_size = 8

source_lang = "[javanese]"
target_lang = "[indonesian]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/jv-id/example'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:
train_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/train_preprocess.json'
valid_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/valid_preprocess.json'
test_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/test_preprocess.json'

backtrans_id_path = '/content/drive/MyDrive/11-737/FinalProject/data/backtrans_ind.json'

train_dataset_flores = MachineTranslationDataset(train_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset_flores = MachineTranslationDataset(valid_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset_flores = MachineTranslationDataset(test_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

backtrans_id_dataset = MachineTranslationDataset(backtrans_id_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_dataset = ConcatDataset([train_dataset_flores, backtrans_id_dataset])

train_loader = GenerationDataLoader(dataset=train_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/drive/MyDrive/11-737/FinalProject/models/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:2.1874 LR:0.00010000: 100%|██████████| 6149/6149 [13:38<00:00,  7.51it/s]


(Epoch 1) TRAIN LOSS:2.1874 BLEU:58.67 SacreBLEU:60.14 ROUGE1:68.55 ROUGE2:51.51 ROUGEL:67.71 ROUGELsum:67.72 LR:0.00010000


VALID LOSS:3.5578: 100%|██████████| 125/125 [00:06<00:00, 20.40it/s]


(Epoch 1) VALID LOSS:3.5578 BLEU:22.26 SacreBLEU:22.49 ROUGE1:47.54 ROUGE2:23.38 ROUGEL:45.63 ROUGELsum:45.61


(Epoch 2) TRAIN LOSS:1.4518 LR:0.00009000: 100%|██████████| 6149/6149 [13:32<00:00,  7.56it/s]


...

(Epoch 9) VALID LOSS:4.3539 BLEU:21.38 SacreBLEU:21.57 ROUGE1:44.81 ROUGE2:21.93 ROUGEL:42.87 ROUGELsum:42.88
count stop: 4


(Epoch 10) TRAIN LOSS:0.1389 LR:0.00003874: 100%|██████████| 6149/6149 [13:34<00:00,  7.55it/s]


(Epoch 10) TRAIN LOSS:0.1389 BLEU:92.59 SacreBLEU:92.85 ROUGE1:96.17 ROUGE2:92.52 ROUGEL:96.14 ROUGELsum:96.14 LR:0.00003874


VALID LOSS:4.4129: 100%|██████████| 125/125 [00:06<00:00, 20.65it/s]


(Epoch 10) VALID LOSS:4.4129 BLEU:21.47 SacreBLEU:21.71 ROUGE1:45.43 ROUGE2:22.04 ROUGEL:43.37 ROUGELsum:43.38
count stop: 5


In [ ]:
! cp '/content/indonlg/save/jv-id/example/best_model_0.th' '/content/drive/My Drive/best_model_jv_id_backtrans_id.th'

In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_jv_id_backtrans_id.th'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 127/127 [01:47<00:00,  1.19it/s]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_flores_jv-id_backtrans-id.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_flores_jv-id_backtrans-id.csv")

== Prediction Result ==
                                                 hyp  \
0  " k ini memiliki desain yang mirip dengan diab...   
1  dr. eh ud ur, profesor ilmu kedokteran di univ...   
2  seperti beberapa ahli yang dia yakin bahwa dia...   
3  pada hari senin, sara dani us, sekretaris teta...   
4  dani us berkata, " k ini kita tidak melakukan ...   

                                               label  
0  “ sa at ini ada menc it umur 4 bulan n ond ia ...  
1  dr. eh ud ur, profesor kedokteran di dal ho us...  
2  seperti halnya ahli - ah li lain, dia merasa s...  
3  sara dani us, sekretaris permanen komite nobel...  
4  " dani us berkata, " sa at ini kita tidak mela...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.000000   1.000000   1.000000   1.000000   1.000000   1.000000
mean   23.904455  24.139143  49.754527  26.313357  47.649258  47.614037
std          NaN        NaN        NaN        NaN        NaN        

In [ ]:
! cp '/content/indonlg/save/jv-id/example/prediction_result_flores_jv-id_backtrans-id.csv' '/content/drive/My Drive/prediction_result_flores_jv-id_backtrans-id.csv'

### With backtranslated jv and id

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = False
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 4
valid_batch_size = 8
test_batch_size = 8

source_lang = "[javanese]"
target_lang = "[indonesian]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/jv-id/example'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:
train_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/train_preprocess.json'
valid_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/valid_preprocess.json'
test_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/test_preprocess.json'

backtrans_id_path = '/content/drive/MyDrive/11-737/FinalProject/data/backtrans_ind.json'
backtrans_jv_path = '/content/drive/MyDrive/11-737/FinalProject/data/backtrans_jav.json'

train_dataset_flores = MachineTranslationDataset(train_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset_flores = MachineTranslationDataset(valid_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset_flores = MachineTranslationDataset(test_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

backtrans_id_dataset = MachineTranslationDataset(backtrans_id_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
backtrans_jv_dataset = MachineTranslationDataset(backtrans_jv_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_dataset = ConcatDataset([train_dataset_flores, backtrans_jv_dataset, backtrans_id_dataset])

train_loader = GenerationDataLoader(dataset=train_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/drive/MyDrive/11-737/FinalProject/models/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:2.3370 LR:0.00010000: 100%|██████████| 7399/7399 [16:15<00:00,  7.58it/s]


(Epoch 1) TRAIN LOSS:2.3370 BLEU:56.24 SacreBLEU:57.86 ROUGE1:66.66 ROUGE2:48.69 ROUGEL:65.72 ROUGELsum:65.72 LR:0.00010000


VALID LOSS:3.3355: 100%|██████████| 125/125 [00:06<00:00, 20.60it/s]


(Epoch 1) VALID LOSS:3.3355 BLEU:21.93 SacreBLEU:22.11 ROUGE1:47.51 ROUGE2:23.00 ROUGEL:45.44 ROUGELsum:45.46


(Epoch 2) TRAIN LOSS:1.5318 LR:0.00009000: 100%|██████████| 7399/7399 [16:11<00:00,  7.61it/s]


...

(Epoch 6) VALID LOSS:4.0163 BLEU:20.28 SacreBLEU:20.46 ROUGE1:44.27 ROUGE2:20.51 ROUGEL:42.02 ROUGELsum:42.05
count stop: 4


(Epoch 7) TRAIN LOSS:0.2595 LR:0.00005314: 100%|██████████| 7399/7399 [16:17<00:00,  7.57it/s]


(Epoch 7) TRAIN LOSS:0.2595 BLEU:87.15 SacreBLEU:87.60 ROUGE1:93.50 ROUGE2:87.33 ROUGEL:93.39 ROUGELsum:93.39 LR:0.00005314


VALID LOSS:4.1432: 100%|██████████| 125/125 [00:06<00:00, 19.70it/s]


(Epoch 7) VALID LOSS:4.1432 BLEU:20.01 SacreBLEU:20.19 ROUGE1:43.93 ROUGE2:20.26 ROUGEL:41.62 ROUGELsum:41.65
count stop: 5


In [ ]:
! cp '/content/indonlg/save/jv-id/example/best_model_0.th' '/content/drive/My Drive/best_model_jv_id_backtrans_jvid.th'

In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_jv_id_backtrans_jvid.th'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 127/127 [01:47<00:00,  1.18it/s]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_flores_jv-id_backtrans-jvid.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_flores_jv-id_backtrans-jvid.csv")

== Prediction Result ==
                                                 hyp  \
0  " sa iki kita memiliki seekor tikus berumur - ...   
1  dr. eh ud ur, profesor ilmu kedokteran di univ...   
2  seperti beberapa ahli lainnya, ia tidak yakin ...   
3  pada hari senin, sara dani us, sekretaris untu...   
4  dani us berkata, " sa iki kita tidak melakukan...   

                                               label  
0  “ sa at ini ada menc it umur 4 bulan n ond ia ...  
1  dr. eh ud ur, profesor kedokteran di dal ho us...  
2  seperti halnya ahli - ah li lain, dia merasa s...  
3  sara dani us, sekretaris permanen komite nobel...  
4  " dani us berkata, " sa at ini kita tidak mela...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.000000   1.000000   1.000000   1.000000   1.000000   1.000000
mean   23.641987  23.791709  48.940754  26.048105  46.776813  46.764536
std          NaN        NaN        NaN        NaN        NaN        

In [ ]:
! cp '/content/indonlg/save/jv-id/example/prediction_result_flores_jv-id_backtrans-jvid.csv' '/content/drive/My Drive/prediction_result_flores_jv-id_backtrans-jvid.csv'

## id-jv

### With backtranslated jv

In [ ]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = True
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 4
valid_batch_size = 8
test_batch_size = 8

source_lang = "[indonesian]"
target_lang = "[javanese]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/id-jv/example'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [ ]:
train_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/train_preprocess.json'
valid_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/valid_preprocess.json'
test_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/test_preprocess.json'

backtrans_id_path = '/content/drive/MyDrive/11-737/FinalProject/data/backtrans_ind.json'
backtrans_jv_path = '/content/drive/MyDrive/11-737/FinalProject/data/backtrans_jav.json'

train_dataset_flores = MachineTranslationDataset(train_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset_flores = MachineTranslationDataset(valid_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset_flores = MachineTranslationDataset(test_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

backtrans_id_dataset = MachineTranslationDataset(backtrans_id_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
backtrans_jv_dataset = MachineTranslationDataset(backtrans_jv_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_dataset = ConcatDataset([train_dataset_flores, backtrans_jv_dataset])

train_loader = GenerationDataLoader(dataset=train_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [ ]:
torch.cuda.empty_cache() 

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/drive/MyDrive/11-737/FinalProject/models/best_model_0.th'))

<All keys matched successfully>

In [ ]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:2.4861 LR:0.00010000: 100%|██████████| 6149/6149 [27:17<00:00,  3.76it/s]


(Epoch 1) TRAIN LOSS:2.4861 BLEU:53.67 SacreBLEU:54.93 ROUGE1:62.67 ROUGE2:43.63 ROUGEL:61.47 ROUGELsum:61.46 LR:0.00010000


VALID LOSS:3.8571: 100%|██████████| 125/125 [00:13<00:00,  9.54it/s]


(Epoch 1) VALID LOSS:3.8571 BLEU:16.13 SacreBLEU:16.05 ROUGE1:40.07 ROUGE2:16.94 ROUGEL:38.03 ROUGELsum:38.04


(Epoch 2) TRAIN LOSS:1.6719 LR:0.00009000: 100%|██████████| 6149/6149 [27:02<00:00,  3.79it/s]


...

(Epoch 6) VALID LOSS:4.6224 BLEU:15.79 SacreBLEU:15.80 ROUGE1:39.19 ROUGE2:16.41 ROUGEL:37.15 ROUGELsum:37.16
count stop: 4


(Epoch 7) TRAIN LOSS:0.2812 LR:0.00005314: 100%|██████████| 6149/6149 [27:11<00:00,  3.77it/s]


(Epoch 7) TRAIN LOSS:0.2812 BLEU:86.86 SacreBLEU:87.26 ROUGE1:92.45 ROUGE2:85.75 ROUGEL:92.31 ROUGELsum:92.31 LR:0.00005314


VALID LOSS:4.7366: 100%|██████████| 125/125 [00:13<00:00,  9.40it/s]


(Epoch 7) VALID LOSS:4.7366 BLEU:16.09 SacreBLEU:16.16 ROUGE1:40.55 ROUGE2:17.08 ROUGEL:38.37 ROUGELsum:38.33
count stop: 5


In [ ]:
! cp '/content/indonlg/save/id-jv/example/best_model_0.th' '/content/drive/My Drive/best_model_id_jv_backtrans_jv.th'

In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_id_jv_backtrans_jv.th'))

<All keys matched successfully>

In [ ]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 127/127 [03:37<00:00,  1.71s/it]


In [ ]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_flores_id-jv_backtrans-jv.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_flores_id-jv_backtrans-jv.csv")

== Prediction Result ==
                                                 hyp  \
0  " sa at iki ana menc it umur 4 wulan n ond ia ...   
1  dr. eh ud ur, profesor kadh okter an ing dal h...   
2  kaya ahli - ah li liya, dh <0xC3> <0xA8> w <0x...   
3  sara dani us, sekretaris permanen komite nobel...   
4  " dani us ngomong, " sa at iki kita ora nindak...   

                                               label  
0  " sa iki kita nduweni tikus umur - 4 - s asi s...  
1  dr. eh ud ur, profesor ilmu kedokteran ing uni...  
2  kaya sawetara ahli liyane, dheweke ora yakin a...  
3  ing dina senin, sara dani us, sekretaris tetep...  
4  dani us ngomong, “ sa iki kita ora nindakake a...  

== Model Performance ==
           BLEU  SacreBLEU     ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.00000   1.000000   1.000000   1.000000   1.000000   1.000000
mean   12.82044  12.950154  34.767972  16.271072  33.438214  33.384745
std         NaN        NaN        NaN        NaN        NaN        NaN


In [ ]:
! cp '/content/indonlg/save/id-jv/example/prediction_result_flores_id-jv_backtrans-jv.csv' '/content/drive/My Drive/prediction_result_flores_id-jv_backtrans-jv.csv'

### With backtranslated id

In [11]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = True
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 4
valid_batch_size = 8
test_batch_size = 8

source_lang = "[indonesian]"
target_lang = "[javanese]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/id-jv/example'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [12]:
train_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/train_preprocess.json'
valid_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/valid_preprocess.json'
test_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/test_preprocess.json'

backtrans_id_path = '/content/drive/MyDrive/11-737/FinalProject/data/backtrans_ind.json'
backtrans_jv_path = '/content/drive/MyDrive/11-737/FinalProject/data/backtrans_jav.json'

train_dataset_flores = MachineTranslationDataset(train_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset_flores = MachineTranslationDataset(valid_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset_flores = MachineTranslationDataset(test_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

backtrans_id_dataset = MachineTranslationDataset(backtrans_id_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
backtrans_jv_dataset = MachineTranslationDataset(backtrans_jv_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_dataset = ConcatDataset([train_dataset_flores, backtrans_id_dataset])

train_loader = GenerationDataLoader(dataset=train_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [13]:
torch.cuda.empty_cache() 

In [14]:
# Load best model
model.load_state_dict(torch.load('/content/drive/MyDrive/11-737/FinalProject/models/best_model_0.th'))

<All keys matched successfully>

In [15]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:2.2667 LR:0.00010000: 100%|██████████| 6149/6149 [13:05<00:00,  7.83it/s]


(Epoch 1) TRAIN LOSS:2.2667 BLEU:55.69 SacreBLEU:56.86 ROUGE1:65.65 ROUGE2:47.14 ROUGEL:64.48 ROUGELsum:64.48 LR:0.00010000


VALID LOSS:4.4299: 100%|██████████| 125/125 [00:05<00:00, 22.10it/s]


(Epoch 1) VALID LOSS:4.4299 BLEU:14.10 SacreBLEU:14.09 ROUGE1:36.49 ROUGE2:15.11 ROUGEL:34.91 ROUGELsum:34.90


(Epoch 2) TRAIN LOSS:1.4130 LR:0.00009000: 100%|██████████| 6149/6149 [13:02<00:00,  7.86it/s]


...

(Epoch 9) VALID LOSS:5.9777 BLEU:14.47 SacreBLEU:14.42 ROUGE1:35.57 ROUGE2:14.72 ROUGEL:33.94 ROUGELsum:33.93


(Epoch 10) TRAIN LOSS:0.0572 LR:0.00003874: 100%|██████████| 6149/6149 [13:02<00:00,  7.86it/s]


(Epoch 10) TRAIN LOSS:0.0572 BLEU:96.79 SacreBLEU:96.88 ROUGE1:98.80 ROUGE2:97.53 ROUGEL:98.79 ROUGELsum:98.78 LR:0.00003874


VALID LOSS:6.0147: 100%|██████████| 125/125 [00:05<00:00, 22.13it/s]


(Epoch 10) VALID LOSS:6.0147 BLEU:14.01 SacreBLEU:13.96 ROUGE1:34.69 ROUGE2:14.20 ROUGEL:33.19 ROUGELsum:33.14
count stop: 1


In [16]:
! cp '/content/indonlg/save/id-jv/example/best_model_0.th' '/content/drive/My Drive/best_model_id_jv_backtrans_id.th'

In [17]:
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_id_jv_backtrans_id.th'))

<All keys matched successfully>

In [18]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 127/127 [01:58<00:00,  1.07it/s]


In [19]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_flores_id-jv_backtrans-jv.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_flores_id-jv_backtrans-jv.csv")

== Prediction Result ==
                                                 hyp  \
0  " sa at iki ana menc it umur 4 wulan n ond ia ...   
1  dr. eh ud ur, profesor ked h okter an ing dal ...   
2  kados ingkang ugi ahli - ah li san <0xC3> <0xA...   
3  sara dani us, sekretaris permanen komite nobel...   
4  " dani us berkata, " sa at punika kita boten m...   

                                               label  
0  " sa iki kita nduweni tikus umur - 4 - s asi s...  
1  dr. eh ud ur, profesor ilmu kedokteran ing uni...  
2  kaya sawetara ahli liyane, dheweke ora yakin a...  
3  ing dina senin, sara dani us, sekretaris tetep...  
4  dani us ngomong, “ sa iki kita ora nindakake a...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.000000    1.00000   1.000000   1.000000   1.000000   1.000000
mean   11.884181   12.01754  31.732388  14.217161  30.413609  30.388497
std          NaN        NaN        NaN        NaN        NaN        

In [20]:
! cp '/content/indonlg/save/id-jv/example/prediction_result_flores_id-jv_backtrans-id.csv' '/content/drive/My Drive/prediction_result_flores_id-jv_backtrans-id.csv'

cp: cannot stat '/content/indonlg/save/id-jv/example/prediction_result_flores_id-jv_backtrans-id.csv': No such file or directory


### With backtranslated jv and id

In [21]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = True
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 4
valid_batch_size = 8
test_batch_size = 8

source_lang = "[indonesian]"
target_lang = "[javanese]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = '/content/indonlg/save/id-jv/example'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [22]:
train_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/train_preprocess.json'
valid_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/valid_preprocess.json'
test_dataset_flores_path = '/content/drive/MyDrive/11-737/FinalProject/data/jv-id/flores/test_preprocess.json'

backtrans_id_path = '/content/drive/MyDrive/11-737/FinalProject/data/backtrans_ind.json'
backtrans_jv_path = '/content/drive/MyDrive/11-737/FinalProject/data/backtrans_jav.json'

train_dataset_flores = MachineTranslationDataset(train_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset_flores = MachineTranslationDataset(valid_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset_flores = MachineTranslationDataset(test_dataset_flores_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

backtrans_id_dataset = MachineTranslationDataset(backtrans_id_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
backtrans_jv_dataset = MachineTranslationDataset(backtrans_jv_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_dataset = ConcatDataset([train_dataset_flores, backtrans_jv_dataset, backtrans_id_dataset])

train_loader = GenerationDataLoader(dataset=train_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset_flores, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

In [23]:
torch.cuda.empty_cache() 

In [24]:
# Load best model
model.load_state_dict(torch.load('/content/drive/MyDrive/11-737/FinalProject/models/best_model_0.th'))

<All keys matched successfully>

In [25]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:2.4394 LR:0.00010000: 100%|██████████| 7399/7399 [15:35<00:00,  7.91it/s]


(Epoch 1) TRAIN LOSS:2.4394 BLEU:53.64 SacreBLEU:54.93 ROUGE1:63.47 ROUGE2:44.18 ROUGEL:62.23 ROUGELsum:62.23 LR:0.00010000


VALID LOSS:3.7672: 100%|██████████| 125/125 [00:05<00:00, 21.97it/s]


(Epoch 1) VALID LOSS:3.7672 BLEU:16.50 SacreBLEU:16.35 ROUGE1:41.28 ROUGE2:17.52 ROUGEL:39.16 ROUGELsum:39.15


(Epoch 2) TRAIN LOSS:1.5975 LR:0.00009000: 100%|██████████| 7399/7399 [15:31<00:00,  7.95it/s]


...

(Epoch 6) VALID LOSS:4.6121 BLEU:15.74 SacreBLEU:15.75 ROUGE1:39.38 ROUGE2:16.45 ROUGEL:37.44 ROUGELsum:37.45
count stop: 4


(Epoch 7) TRAIN LOSS:0.2570 LR:0.00005314: 100%|██████████| 7399/7399 [15:26<00:00,  7.99it/s]


(Epoch 7) TRAIN LOSS:0.2570 BLEU:88.03 SacreBLEU:88.40 ROUGE1:93.19 ROUGE2:87.14 ROUGEL:93.07 ROUGELsum:93.06 LR:0.00005314


VALID LOSS:4.7924: 100%|██████████| 125/125 [00:05<00:00, 22.41it/s]


(Epoch 7) VALID LOSS:4.7924 BLEU:15.73 SacreBLEU:15.76 ROUGE1:39.89 ROUGE2:16.59 ROUGEL:37.59 ROUGELsum:37.59
count stop: 5


In [26]:
! cp '/content/indonlg/save/id-jv/example/best_model_0.th' '/content/drive/My Drive/best_model_id_jv_backtrans_jvid.th'

In [27]:
model.load_state_dict(torch.load('/content/drive/My Drive/best_model_id_jv_backtrans_jvid.th'))

<All keys matched successfully>

In [28]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 127/127 [01:55<00:00,  1.10it/s]


In [29]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_flores_id-jv_backtrans-jvid.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_flores_id-jv_backtrans-jvid.csv")

== Prediction Result ==
                                                 hyp  \
0  " sa at iki ana menc it umur 4 wulan n ond ia ...   
1  dr. eh ud ur, profesor kadh okter an ing dal h...   
2  kados ta ahli - ah li san <0xC3> <0xA8> sipun,...   
3  sara dani us, sekretaris permanen komite nobel...   
4  " dani us ujar, " sa at punika kita boten nind...   

                                               label  
0  " sa iki kita nduweni tikus umur - 4 - s asi s...  
1  dr. eh ud ur, profesor ilmu kedokteran ing uni...  
2  kaya sawetara ahli liyane, dheweke ora yakin a...  
3  ing dina senin, sara dani us, sekretaris tetep...  
4  dani us ngomong, “ sa iki kita ora nindakake a...  

== Model Performance ==
            BLEU  SacreBLEU     ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.000000    1.00000   1.000000   1.000000   1.000000   1.000000
mean   11.759703   11.91569  31.447577  14.089346  30.227456  30.253029
std          NaN        NaN        NaN        NaN        NaN        

In [30]:
! cp '/content/indonlg/save/id-jv/example/prediction_result_flores_id-jv_backtrans-jvid.csv' '/content/drive/My Drive/prediction_result_flores_id-jv_backtrans-jvid.csv'